Defining global path to python. In this case these variables will be stored after one execution

In [5]:
SRC='/home/jmurga/mkt/201902/scripts/src'
DATA='/home/jmurga/mkt/201902/rawData'

Required libraries

In [6]:
SRC='/home/jmurga/mkt/201902/scripts/src'
DATA='/home/jmurga/mkt/201902/rawData'
import os
import re
import sys
import numpy as np
import pandas as pd
from numpy import array 
from pyfaidx import Fasta
sys.path.insert(0, SRC)
from reverseComplement import reverseComplement
from degenerate import degenerate
from dafResampling import dafWithResampling

### Drosophila melanogaster genes 

To execute bash code we created a snippets through nbextensions containing the following paths in order to avoid copy and paste cell contains. %%bash magic do not recognize previous variables. Adding mkdir command to create necesary paths

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
BASIC='/home/jmurga/mkt/201902/rawData/annotations/basicAnnotation'
CDS='/home/jmurga/mkt/201902/rawData/annotations/cds'
GENES='/home/jmurga/mkt/201902/rawData/annotations/genes'
TEMPORAL='/home/jmurga/mkt/201902/rawData/annotations/tmp'

mkdir -p ${DATA}
mkdir -p ${BASIC}
mkdir -p ${CDS}
mkdir -p ${GENES}
mkdir -p ${TEMPORAL}

#### Download Flybase annotation

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'

cd ${DATA}
# Flybase annotation
wget ftp://ftp.flybase.net/genomes/Drosophila_melanogaster/dmel_r5.57_FB2014_03/gff/dmel-all-no-analysis-r5.57.gff.gz
gunzip dmel-all-no-analysis-r5.57.gff.gz

#### Parsing and cleaning flybase annotation to execute all operations by chr
Next operation will be faster due to grep on smaller files. Space is not a problem in our server right now. Each folder contain an specific file foreach chromosome. 

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
BASIC='/home/jmurga/mkt/201902/rawData/annotations/basicAnnotation'
CDS='/home/jmurga/mkt/201902/rawData/annotations/cds'
GENES='/home/jmurga/mkt/201902/rawData/annotations/genes'
TEMPORAL='/home/jmurga/mkt/201902/rawData/annotations/tmp'

mkdir -p ${BASIC}
mkdir -p ${CDS}
mkdir -p ${GENES}
mkdir -p ${TEMPORAL}

# Extract sequences f
# Extract annotations from gff file
sed -e '/^>/,$d' ${DATA}/dmel-all-no-analysis-r5.57.gff | sed -e '/^2LHet/d' -e  '/2RHet/d' -e '/3LHet/d' -e '/3RHet/d' -e  '/^4\t/d' -e '/dmel_mitochondrion_genome/d' -e '/^U\t/d' -e '/^Uextra\t/d' -e  '/^XHet\t/d' -e '/^YHet\t/d' -e'/\tCG/d' > ${DATA}/dmelFiltered.gff
# Extract genes information
grep -P "\tgene\t" ${DATA}/dmelFiltered.gff  > ${DATA}/dmelFilteredGenes.gff
# Extract CDS information. Only protein coding genes
grep -P "\tCDS\t" ${DATA}/dmelFiltered.gff  > ${DATA}/dmelFilteredCds.gff
# Coding gene list. gene_id always on column 9, 3th field
cut -f1,9 ${DATA}/dmelFilteredCds.gff | tr ';' '\t' | cut -f1,2 | sort -u | sort -k1,1 > ${DATA}/codingGeneList.txt


CHR=( 2L 2R 3L 3R X )
# Parse gff file by chr
for chrNumber in "${CHR[@]}"
do
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredGenes.gff | sort -k4,4n > ${BASIC}/gencode.v27lift37.basic.annotation.chr${chrNumber}.gff3 
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredCds.gff | sort -k1,1 -k4,4n > ${CDS}/dmelFilteredCdsChr${chrNumber}.gff3 
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredGenes.gff | sort -k1,1 -k4,4n > ${GENES}/dmelFilteredGenesChr${chrNumber}.gff3 
done

rm ${DATA}/dmel-all-no-analysis-r5.57.gff

#### Basic cleaned gene file
This file will include information about chromosomes, start coordinates, end coordinates strand, gene id and gene name. It will be and perform calculations on gene coordinates. Kind of gff file easier to work with

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
GENES='/home/jmurga/mkt/201902/rawData/annotations/genes'
touch ${DATA}/flybaseGenesCleaned.tab 
printf "chr\tstartGene\tendGene\tstrand\tid\tname\n" > ${DATA}/flybaseGenesCleaned.tab

time while read LINE;
do 
    CHR=$(echo ${LINE} | cut -d' ' -f1)
    GENE=$(echo ${LINE} | cut -d' ' -f2 | sed 's/-cds//g')
    # echo $GENE

    fgrep `echo "${GENE};"`  ${GENES}/dmelFilteredGenesChr${CHR}.gff3 | fgrep ${CHR} |  cut -f1,4,5,7,9 | tr ';' '\t' | cut -f1,2,3,4,5,6 
    
done < ${DATA}/codingGeneList.txt | tr ' ' '\t' | sed 's/ID=//g' | sed 's/Name=//g' | sort -k1,1 -k2,2n >> ${DATA}/flybaseGenesCleaned.tab

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',header = 0,sep='\t')
dfGenes.head()

File to merge features annotated with gene name instead of gene id

In [ ]:
idName = dfGenes[['id','name']]
idName.to_csv(DATA + '/annotations/idName.tab',sep='\t',index=False,header=True)

### Extracting CDS coordinates from largest transcript by gene
From the largest transcript we recover id, size, and CDS coordinates. Moreover we save number of transcripts too.  
Coordinates will be used to recover CDS sequences and to calcute Derived Allele Frequencies from 0fold and 4fold degenerate sequences

#### Raw cds coordinates

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
CDS='/home/jmurga/mkt/201902/rawData/annotations/cds'
TEMPORAL='/home/jmurga/mkt/201902/rawData/annotations/tmp'

touch ${DATA}/cdsCoordinates.tab
printf "name\tchr\ttranscript\ttranscriptSize\tcoordinates\n" > ${DATA}/cdsCoordinates.tab

count=0
time tail -n+2 ${DATA}/flybaseGenesCleaned.tab | while read LINE;
do 
    echo '*************'    
    CHR=$(echo ${LINE} | cut -d' ' -f1)
    GENE=$(echo ${LINE} | cut -d' ' -f6)
    
    printf "${GENE} ${CHR}"
    # fgrep ${GENE} ${CDS}/dmelFilteredCdsChr${CHR}.gff3 | cut -f9 | cut -d';' -f2 | cut -d'=' -f2 | sort -u > ${TEMPORAL}/transcriptTmp.tab
    
    fgrep `echo "Name=${GENE}-cds;"` ${CDS}/dmelFilteredCdsChr${CHR}.gff3 | grep -P "${CHR}\t" | cut -f9 | tr ';' '\n' | fgrep Parent | sort -u | tr ',' '\n' | cut -d'=' -f2 | sort -u > ${TEMPORAL}/transcriptTmp.tab

    while read transcript; do fgrep ${transcript} ${CDS}/dmelFilteredCdsChr${CHR}.gff3 | awk '{print $4,$5,$5-$4}' | awk -v gene="${GENE}" -v chr="${CHR}" -v transcript="$transcript" '{sum+=$1} {printf $1","$2","} END{print "\t"gene"\t"chr"\t"transcript"\t"sum}' | awk '{print $2,$3,$4,$5,$1}' | sed 's/,$//' |tr ' ' '\t' >> ${DATA}/cdsCoordinates.tab ;done < ${TEMPORAL}/transcriptTmp.tab 
    
#     paste <(echo $GENE) <(echo $CHR) <(wc -l ${TEMPORAL}/transcriptTmp.tab | cut -d' ' -f1) <(echo $largestTranscript | cut -d' ' -f1) <(echo $largestTranscript | cut -d' ' -f2) <(fgrep `echo ${largestTranscript} | cut -d' ' -f1` ${CDS}/dmelFilteredCdsChr${CHR}.gff3  | cut -f4,5 | sort -k1,1n | awk '{printf $1","$2","}' | sed 's/.$//') >> ${DATA}/cdsCoordinates.tab
    
    (( count++ ))

done 


Recode all cds positions and aggregate all positions to annotate all posible functional duplicated sites as 0fold if required

In [ ]:
sys.path.insert(0, SRC)
from reverseComplement import reverseComplement
from degenerancy import degenerate

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',header = 0,usecols=['id','name','chr','strand'],sep='\t')
cds = pd.read_csv(DATA + '/annotations/cdsCoordinates.tab',header=0,sep='\t')
cds = pd.merge(cds, dfGenes,  how='inner', left_on=['chr','name'], right_on = ['chr','name'])

In [ ]:
# Rewrite file each execution
total = []
chrNumber = cds['chr'].unique()
import time
for j in chrNumber:
    start_time = time.time()
    degeneratePositions = pd.DataFrame()
    print(j)
    for index, row in cds[cds['chr']==j].iterrows():
#         print(index,row['id'])
        chrFile = Fasta(DATA + '/fastas/ref/Chr' + row['chr'] +'.fasta')
        # Convert CDS list into numeric array
        coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
        coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
        # Extract all CDS positions in a list in order to merge with degenerate sequences (same length -> same index)
        positions=[]
        for i in range(0,len(coordinates),1):
            positions.append(list(range(coordinates[i][0],coordinates[i][1]+1)))  
        allPositions = [item for sublist in positions for item in sublist]
        # Extract cds sequences
        seq = chrFile.get_spliced_seq(row['chr'].replace('chr',''), coordinates).seq.upper()
    #     print(len(seq))
        if(row['strand'] == '-'):
            seq = reverseComplement(seq)
            allPositions = allPositions[::-1]
        if((len(seq)/3).is_integer() and seq[0:3]=='ATG'):
            m = degenerate(seq)
            degenerateDF = pd.DataFrame({'m':list(m),'POS':allPositions})
            degeneratePositions = pd.concat([degeneratePositions,degenerateDF])
    total.append(degeneratePositions)
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
for i in range(0,len(total),1): 
    print(i)
    tmp = total[i].groupby(['chr','POS'])['m'].apply(lambda x: ','.join(x)).reset_index()
    tmp.to_csv('/home/jmurga/degeneracyDmelPositions.tab',mode='a',index=False,sep='\t')
#     totalGrouped.append(tmp) 

Cleaning positions degenerancy based on most constrain posibility

In [ ]:
degeneracyPositions = pd.read_csv('/home/jmurga/positiveSelectionHuman/201901/rawData/dmel/degeneracyDmelPositions.tab',sep='\t',header=0)
degeneracyPositions['newType'] = np.nan

In [ ]:
def foldPositions(x):
    if('0' in x):
        return('0fold')
    elif('4' in x and '0' not in x and '2' not in x and '3' not in x):
        return('4fold')
    elif('2' in x and '0' not in x):
        return('2fold')
    elif('2' not in x and '0' not in x and '4' not in x) :
        return('3fold')
    else:
        return(x)

In [ ]:
degeneracyPositions['newType'] = degeneracyPositions['type'].apply(foldPositions)

In [ ]:
df = degeneracyPositions[(degeneracyPositions['newType']=='0fold') | (degeneracyPositions['newType']=='4fold')]

In [ ]:
df.to_csv('/home/jmurga/positiveSelectionHuman/201901/rawData/dmel/zeroFourFoldPositions.tab',header=True,index=False,sep='\t')

#### N calls distributions

##### N calls distribution by lines

In [ ]:
nCallsX = []

for index, row in cds[cds['chr']=='X'].iterrows():
    print(index)
    # Convert CDS list into numeric array
    coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
    coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
    
    
    # Open ref and outgroup
    ref = Fasta(DATA + '/fastas/ref/Chr' + row['chr'] +'.fasta')  
    ## Extract ref and outgroup seq
    refSeq = ref.get_spliced_seq(row['chr'], coordinates).seq.upper()    

    if(('N' not in refSeq) and (len(refSeq)/3).is_integer()):

        # Open population multifasta
        popFasta = Fasta(DATA + '/fastas/alignments/RAL/' + 'RAL_' + 'Chr' + row['chr'] +'.seq')

        #Extract samples
        samples = list(popFasta.keys())

        deletedIndex = 0
        if(row['strand'] == '-'):            

            for i in range(0,len(samples),1):
                tmp = popFasta.get_spliced_seq(samples[i], coordinates).seq.upper()
                tmp = reverseComplement(tmp)
                if(tmp == ('N' * len(refSeq))):
                    deletedIndex+=1

        else:

            for i in range(0,len(samples),1):
                tmp = popFasta.get_spliced_seq(samples[i], coordinates).seq.upper()
                if(tmp == 'N' * len(tmp)):
                    # Save lines index with only N
                    deletedIndex+=1
        nCallsX.append(deletedIndex)
    else:
        continue

In [ ]:
nCalls = pd.DataFrame(nCalls).value_counts().reset_index()

##### N call distrubitution by position

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',header = 0,sep='\t')

In [ ]:
nCallsByPositions = []
for index, row in dfGenes.iterrows():    
# for index, row in cds.iterrows():
    print(row['id'])
    # Convert CDS list into numeric array
#     coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
#     coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
    # Open ref and outgroup
    ref = Fasta(DATA + '/fastas/ref/Chr' + row['chr'] +'.fasta')  
    ## Extract ref and outgroup seq
#     refSeq = ref.get_spliced_seq(row['chr'],[row['startGene'],row['endGene']]).seq.upper()    
    refSeq = ref.get_seq(row['chr'],row['startGene'],row['endGene']).seq.upper()    
#     if(('N' not in refSeq) and (len(refSeq)/3).is_integer()):
    if('N' not in refSeq):
        # Open population multifasta
        popFasta = Fasta(DATA + '/fastas/alignments/RAL/' + 'RAL_' + 'Chr' + row['chr'] +'.seq')
        #Extract samples
        samples = list(popFasta.keys())
        matrixDna = np.empty([len(samples)+1,len(refSeq)],dtype='str')
        if(row['strand'] == '-'):            
            refSeq = reverseComplement(refSeq)
            matrixDna[0] = list(refSeq)
            for i in range(0,len(samples),1):
#                 tmp = popFasta.get_spliced_seq(samples[i], coordinates).seq.upper()
                tmp = popFasta.get_seq(samples[i], row['startGene'],row['endGene']).seq.upper()
                tmp = reverseComplement(tmp)
                matrixDna[i+1] = list(tmp)
        else:
            matrixDna[0] = list(refSeq)
            for i in range(0,len(samples),1):
                tmp = popFasta.get_seq(samples[i],row['startGene'],row['endGene']).seq.upper()
                matrixDna[i+1] = list(tmp)
        # Count occurences
        df = pd.DataFrame(matrixDna)
        for i in df.columns:
            nCallsByPositions.append(df[i].value_counts()['N'])
    else:
        continue

In [ ]:
nCalls = pd.Series(nCallsByPositions).value_counts().reset_index()
nCalls = pd.DataFrame(nCalls)

In [ ]:
nCalls.to_csv('/home/jmurga/mkt/201902/rawData/nCallsByPositionsAllGenes.tab')

#### SFS, Pi and P0

Opening cdsCoordinates file to extract fasta sequence using pyfaidx

In [10]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --help

usage: sfsDivergence.py [-h] --genes GENES --cds CDS --outgroup {dsim,dyak}
                        --population POPULATION --sampling SAMPLING
                        [--seed SEED] [--path PATH]

Extract alleles frequencies from multi-FASTA aligment

optional arguments:
  -h, --help            show this help message and exit
  --genes GENES         File basic gene information.
  --cds CDS             File all transcript coordinates by genes.
  --outgroup {dsim,dyak}
                        Select outgroup to compute diverenge and derived
                        allele frequency
  --population POPULATION
                        Select population to extract
  --sampling SAMPLING   Resampling size
  --seed SEED           Input seed
  --path PATH           Path to output file


##### American populations

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population RAL --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population USI --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population USW --sampling 160 --outgroup dsim

##### African populations

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population CO --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population EA --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population EF --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population EG --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population GA --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population RG --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population SP --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population SD --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population ZI --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population GA --sampling 160 --outgroup dsim

##### Asia

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population CHB --sampling 160 --outgroup dsim

##### Europe

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population FR --sampling 160 --outgroup dsim

In [ ]:
!python /home/jmurga/mkt/201902/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population NTH --sampling 160 --outgroup dsim

##### Raw code

In [ ]:
# Rewrite file each execution
columns = ['id','rawDerivedAllele','type']
dmelDaf = pd.DataFrame(columns=columns)
columns = ['id','div','type']
dmelDiv = pd.DataFrame(columns=columns)
# os.remove('/home/jmurga/mkt/201902/dmelFourFoldSites.tab')
# os.remove('/home/jmurga/mkt/201902/dmelZeroFoldSites.tab')          
import time
start = time.time()
for index, row in cds.head(1).iterrows():
# for index, row in cds.iterrows():
    # Convert CDS list into numeric array
    start_time = time.time()
    print(index,row['id'])
    coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
    coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
    # Open ref and outgroup
    ref = Fasta(DATA + '/fastas/ref/Chr' + row['chr'] +'.fasta')
    outgroup = Fasta(DATA + '/fastas/outgroup/dsim/Chr' + row['chr'] +'_dsim.fasta')
    ## Extract ref and outgroup seq
    refSeq = ref.get_spliced_seq(row['chr'], coordinates).seq.upper()
    outgroupSeq = outgroup.get_spliced_seq('Dsimulans', coordinates).seq.upper()
    if((len(refSeq)/3).is_integer()): 
        # Open population multifasta
        popFasta = Fasta(DATA + '/fastas/alignments/RAL/' + 'RAL_' + 'Chr' + row['chr'] +'.seq')
        #Extract samples
        samples = list(popFasta.keys())
        # Open variables and file to write
    #     multifasta = open(DATA + '/fastas/genes/RAL/' + row['id'] +'.fa', 'w')   
        positions=[]
        for i in range(0,len(coordinates),1):
            positions.append(list(range(coordinates[i][0],coordinates[i][1]+1)))  
        allPositions = [str(item) for sublist in positions for item in sublist]
        ## Empty matrix to append ref, pop
        matrix0f=np.empty([len(samples)+2,len(refSeq)],dtype='str')
        matrix4f=np.empty([len(samples)+2,len(refSeq)],dtype='str')
        # Outgroup sequences to pd.DataFrame
        outgroup = np.empty([1,len(refSeq)],dtype='str')
        if(row['strand'] == '-'):
        #     if((len(refSeq)/3).is_integer()):
            refSeq = reverseComplement(refSeq)
            refSeq0f,refSeq4f = degenerate(refSeq)
            outgroupSeq = reverseComplement(outgroupSeq)
            matrix4f[0] = list(refSeq4f)        
            matrix0f[0] = list(refSeq0f)
            # Iter by row matrix to input sequences
            deleteIndex = []
            for i in range(0,len(samples),1):
                tmp = popFasta.get_spliced_seq(samples[i], coordinates).seq.upper()
                tmp = reverseComplement(tmp)
                if(tmp == ('N' * len(tmp))):
                    deleteIndex.append(i+1)
                else:
                    matrix4f[i+1] = list(tmp)
                    matrix0f[i+1] = list(tmp)
            matrix4f = np.delete(matrix4f,deleteIndex,0)
            matrix0f = np.delete(matrix0f,deleteIndex,0)
        else:
            refSeq0f,refSeq4f = degenerate(refSeq)
            matrix4f[0] = list(refSeq4f)        
            matrix0f[0] = list(refSeq0f)
            # Iter by row matrix to input sequences
            deleteIndex = []
            for i in range(0,len(samples),1):
                tmp = popFasta.get_spliced_seq(samples[i], coordinates).seq.upper()
                if(tmp == ('N' * len(tmp))):
                    deleteIndex.append(i+1)
                else:
                    matrix4f[i+1] = list(tmp)
                    matrix0f[i+1] = list(tmp)

            matrix4f = np.delete(matrix4f,deleteIndex,0)
            matrix0f = np.delete(matrix0f,deleteIndex,0)
            # Iter by matrix column to degenerate all sequences based on reference sequence
        # OutputSeq to pd.DataFrame
        matrix4f[len(matrix4f)-1] = list(outgroupSeq)
        matrix0f[len(matrix4f)-1] = list(outgroupSeq)
        # 0-4fold positions to pd.DataFrame
        df4f = pd.DataFrame(matrix4f)
        df4f.columns = allPositions
        df4f = df4f.loc[:,df4f.iloc[0]!='N']   
        df4f = df4f.transpose()
        
        df0f = pd.DataFrame(matrix0f)
        df0f.columns = allPositions
        df0f = df0f.loc[:,df0f.iloc[0]!='N'] 
        df0f = df0f.transpose()

        dmelFourFoldDafDiv = dafWithResampling(id=row['id'],data=df4f,resamplingValue=160,type='4fold')
        dmelZeroFoldDafDiv = dafWithResampling(id=row['id'],data=df0f,resamplingValue=160,type='0fold')
        
        dmelFourFoldDafDiv.to_csv('/home/jmurga/mkt/201902/dmelFourFoldSites.tab',sep='\t',header=False,mode='a',index=False)
        dmelZeroFoldDafDiv.to_csv('/home/jmurga/mkt/201902/dmelZeroFoldSites.tab',sep='\t',header=False,mode='a',index=False)
        
    else:
        dmelFourFoldDafDiv = pd.DataFrame({'id':row['id'],'POS':0,'rawDerivedAllele':0,'div':0,'type':'4fold'},index=[0])
        dmelZeroFoldDafDiv = pd.DataFrame({'id':row['id'],'POS':0,'rawDerivedAllele':0,'div':0,'type':'0fold'},index=[0])
        dmelFourFoldDafDiv.to_csv('/home/jmurga/mkt/201902/dmelFourFoldSites.tab',sep='\t',header=False,mode='a',index=False)
        dmelZeroFoldDafDiv.to_csv('/home/jmurga/mkt/201902/dmelZeroFoldSites.tab',sep='\t',header=False,mode='a',index=False)
    print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s seconds ---" % (time.time() - start))


#### Extracting mi and m0

In [ ]:
def degenerate(data):
        
        #DEGENERANCY DICTIONARIES
        degenerateCodonTable = {"TTC":'002',"TTT":'002',#PHE 002
                                                "TTA":'002',"TTG":'002',#LEU 002
                                                "CTT":'204',"CTC":'204',"CTA":'204',"CTG":'204',#LUE 204
                                                "ATG":'000',#MET 000
                                                "ATT":'003',"ATC":'003',"ATA":'003',#ILE 003
                                                "GTC":'004',"GTT":'004',"GTA":'004',"GTG":'004',#VAL 004
                                                "TCT":'004',"TCA":'004',"TCC":'004',"TCG":'004',#SER 004
                                                "CCT":'004',"CCA":'004',"CCC":'004',"CCG":'004',#PRO 004
                                                "ACT":'004',"ACA":'004',"ACC":'004',"ACG":'004',#THR 004
                                                "GCT":'004',"GCA":'004',"GCC":'004',"GCG":'004',#ALA 004
                                                "TAT":'002',"TAC":'002',#TYR 002
                                                "TAA":'000',"TAG":'000',"TGA":'000',#STOP 000
                                                "CAT":'002',"CAC":'002',#HIS 002
                                                "CAA":'002',"CAG":'002',#GLN 002
                                                "AAT":'002',"AAC":'002',#ASN 002
                                                "AAG":'002',"AAA":'002',#LYS 002
                                                "GAT":'002',"GAC":'002',#ASP 002
                                                "GAA":'002',"GAG":'002',#GLU 002
                                                "TGT":'002',"TGC":'002',#CYS 002
                                                "TGG":'000',#TRP 000
                                                "CGT":'204',"CGG":'204',"CGA":'204',"CGC":'204',#ARG 204
                                                "AGA":'002',"AGG":'002',#ARG 002
                                                "AGT":'002',"AGC":'002',#SER 002
                                                "GGT":'004',"GGA":'004',"GGC":'004',"GGG":'004',#GLY 004                        
                
        }
        
        degenerancy = ''
        for i in range(0, len(data),3):
                codon = data[i:i+3]
                degenerancy += degenerateCodonTable[codon]
        return(degenerancy)

In [ ]:
dfGenes = pd.read_csv('/home/jmurga/positiveSelectionHuman/201901/rawData/dmel/annotations/flybaseGenesCleaned.tab',header = 0,usecols=['id','name','chr','strand'],sep='\t')
cds = pd.read_csv('/home/jmurga/mkt/201902/rawData/annotations/cdsLargest.tab',header=0,sep='\t')
cds = pd.merge(cds, dfGenes,  how='inner', left_on=['chr','name'], right_on = ['chr','name'])
degeneratePositions = pd.read_csv('/home/jmurga/positiveSelectionHuman/201901/rawData/dmel/annotations/zeroFourFoldPositions.tab',header=0,sep='\t')

In [ ]:
# Rewrite file each execution
columns = ['id','mi','m0']
totalFoldPositions = pd.DataFrame(columns=columns)
for j in cds['chr'].unique():
    print(j)
    chrPositions = degeneratePositions[degeneratePositions['chr']==j]
    for index, row in cds[cds['chr']==j].iterrows():
        print(index,row['id'])
        chrFile = Fasta('/home/jmurga/positiveSelectionHuman/201901/rawData/dmel/refFastas/' + row['chr'] +'.fa')
    #     Convert CDS list into numeric array
        coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
        coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
        # Extract all CDS positions in a list in order to merge with degenerate sequences (same length -> same index)
        positions=[]
        for i in range(0,len(coordinates),1):
            positions.append(list(range(coordinates[i][0],coordinates[i][1]+1)))  
        allPositions = [item for sublist in positions for item in sublist]
        # Extract cds sequences
        seq = chrFile.get_spliced_seq(row['chr'].replace('chr',''), coordinates).seq.upper()
    #     print(len(seq))
        if(row['strand'] == '-'):
            seq = reverseComplement(seq)
            allPositions = allPositions[::-1]
        if((len(seq)/3).is_integer() and seq[0:3]=='ATG'):
            m = degenerate(seq)
            tmp = pd.DataFrame({'pos':allPositions,'m':list(m)})
            tmp['chr']=row['chr']
            tmp = pd.merge(chrPositions,tmp,on=['chr','pos'],how='right')              
            tmp['m']=tmp.apply(lambda x: '0' if x['newType']=='0fold' else '4' if x['newType']=='4fold' else x['m'],axis=1)
            counts = tmp['m'].value_counts()
            if(counts.shape[0]<4 or '4' not in counts.index):
                m0=0
                mi=counts['0']
            else:
                m0=counts['4']
                mi=counts['0']
            data = pd.DataFrame({'id':row['id'],'mi':mi,'m0':m0},index=[0])
            totalFoldPositions = totalFoldPositions.append(data,ignore_index=True)
        else:
            data = pd.DataFrame({'id':row['id'],'mi':0,'m0':0},index=[0])
            totalFoldPositions = totalFoldPositions.append(data,ignore_index=True)